In [12]:
import numpy as np
import copy
from datetime import datetime
import numpy as np
import plotly.figure_factory as ff
import pandas as pd

In [13]:
kapa = 3 #kapazität der Fahrzeuge
M = 1000 
I = [0,1,2,3,4,5] #5 Kunden+Depot = Knoten
K = 2 #Anzahl Fahrzeuge = Anzahl Touren
seed = [] #Vordefinierte Seed Customer
c = np.array([[0,12,24,10,24,25],[12,0,24,21,14,27],
              [24,24,0,32,37,48],[10,21,32,0,30,22],[24,14,37,30,0,25],[25,27,48,22,25,0]]) #Kostenmatrix, hier Distanz in Minuten
startzeitfenster = [0, 20, 10, 50, 55, 70, 0]
endezeitfenster = [100, 40, 35, 70, 65, 75,100]
kundenOffen = copy.deepcopy(I)
kundenOffen.remove(0)

In [14]:
c

array([[ 0, 12, 24, 10, 24, 25],
       [12,  0, 24, 21, 14, 27],
       [24, 24,  0, 32, 37, 48],
       [10, 21, 32,  0, 30, 22],
       [24, 14, 37, 30,  0, 25],
       [25, 27, 48, 22, 25,  0]])

In [15]:
kunden = [i for i in kundenOffen if i not in seed] #Alle Kunden die noch nicht im Tourenplan sind
tourenplaninit = []
tourenplaninit.append([0,0]) #hier manuell vorgegeben, sollte leer sein
tourenplaninit.append([0,0])

In [16]:
tourenplaninit

[[0, 0], [0, 0]]

In [17]:
tourenplan = copy.deepcopy(tourenplaninit)

In [18]:
kunden

[1, 2, 3, 4, 5]

In [19]:
def parinsertion (tourenplan, kunden, M, K, kapa, startzeitfenster, endezeitfenster):
    szf = copy.deepcopy(startzeitfenster)
    ezf = copy.deepcopy(endezeitfenster)
    reject = []
    Kunden = copy.deepcopy(kunden)
    #in jeder Iteration wird ein Kunde aus der Liste genommen, bis diese leer ist
    while Kunden != []:
        pOpt = M #initialisiert mit sehr hoher Zahl M
        for j in Kunden:
            
            #K ist die Anzahl der Touren/Fahrzeuge
            for k in range(K): 
               
                #pos ist die Position in Tour k, an der der Kunde eingefügt wird
                for pos in range(1, len(tourenplan[k])): # iteriert von der zweiten bis zur vorletzen Position der Tour
                    
                    #Feasibility Check: Kann der Kunde an pos eingefügt werden ohne das ein Zeitfenster verpasst wird?    
                    e = max(szf[j],szf[tourenplan[k][pos-1]]+c[tourenplan[k][pos-1],j])
                    l = min(ezf[j],ezf[tourenplan[k][pos]]-c[tourenplan[k][pos],j])
                    if e <= l:
                       
                        kosten = c[tourenplan[k][pos-1],j]+c[j,tourenplan[k][pos]]-c[tourenplan[k][pos-1],tourenplan[k][pos]]#-1 ist letzte position 
                        if  kosten < pOpt: #Verwende die "optimale" Lösung
                            jOpt = j
                            kOpt = k
                            posOpt = pos
                            pOpt = kosten
        if pOpt < M:   
            tourenplan[kOpt].insert(posOpt,jOpt) #Setze denn "optimalen" Kunden, an der "optimalen" Posititon im "optimalen" Tourenplan ein

            #Update die Start- und Endzeitfenster
            for i in range(posOpt, len(tourenplan[kOpt])) :
                szf[tourenplan[kOpt][i]] = max(szf[tourenplan[kOpt][i]], szf[tourenplan[kOpt][i-1]]+c[tourenplan[kOpt][i],i-1])
            for i in range(posOpt-1, 0, -1):
                ezf[tourenplan[kOpt][i]] = min(ezf[tourenplan[kOpt][i]], ezf[tourenplan[kOpt][i+1]]-c[tourenplan[kOpt][i],i+1])
            
            Kunden.remove(jOpt) #Entferne den eigesetzten Kunden aus der Liste der noch ausstehenden Kunden
        
        else:
            reject = copy.deepcopy(Kunden)
            Kunden = []
   
    return tourenplan,reject

In [20]:
opttourenplan, reject  = parinsertion(tourenplaninit, kunden, M, K, kapa, startzeitfenster, endezeitfenster)
opttourenplan, reject

([[0, 3, 5, 0], [0, 0]], [1, 2, 4])

In [ ]:
gesamtkost = 0
for k in range(K):
    tourkost = 0
    for pos in range(len(opttourenplan[k])-1):
        tourkost = tourkost + c[opttourenplan[k][pos],opttourenplan[k][pos+1]]
    gesamtkost = gesamtkost + tourkost 
print(gesamtkost) 

In [ ]:

    szf = copy.deepcopy(startzeitfenster)
    ezf = copy.deepcopy(endezeitfenster)
    print(szf)
    print(ezf)
    reject = []
    Kunden = copy.deepcopy(kunden)
    #in jeder Iteration wird ein Kunde aus der Liste genommen, bis diese leer ist
    while Kunden != []:
        pOpt = M #initialisiert mit sehr hoher Zahl M
        for j in Kunden:
            
            #K ist die Anzahl der Touren/Fahrzeuge
            for k in range(K): 
               
                #pos ist die Position in Tour k, an der der Kunde eingefügt wird
                for pos in range(1, len(tourenplan[k])): # iteriert von der zweiten bis zur vorletzen Position der Tour
                    
                    #Feasibility Check: Kann der Kunde an pos eingefügt werden ohne das ein Zeitfenster verpasst wird?    
                    e = max(szf[j],szf[tourenplan[k][pos-1]]+c[tourenplan[k][pos-1],j])
                    l = min(ezf[j],ezf[tourenplan[k][pos]]-c[tourenplan[k][pos],j])
                    if e <= l:
                       
                        kosten = c[tourenplan[k][pos-1],j]+c[j,tourenplan[k][pos]]-c[tourenplan[k][pos-1],tourenplan[k][pos]]#-1 ist letzte position 
                        if  kosten < pOpt: #Verwende die "optimale" Lösung
                            jOpt = j
                            kOpt = k
                            posOpt = pos
                            pOpt = kosten
        if pOpt < M:   
            tourenplan[kOpt].insert(posOpt,jOpt) #Setze denn "optimalen" Kunden, an der "optimalen" Posititon im "optimalen" Tourenplan ein
            print(tourenplan)
            #Update die Start- und Endzeitfenster
            for i in range(posOpt, len(tourenplan[k])) :
                szf[tourenplan[k][i]] = max(szf[tourenplan[k][i]], szf[tourenplan[k][i-1]]+c[tourenplan[k][i],i-1])
            for i in range(posOpt-1, 0, -1):
                ezf[tourenplan[k][i]] = min(ezf[tourenplan[k][i]], ezf[tourenplan[k][i+1]]-c[tourenplan[k][i],i+1])
            print(szf)
            print(ezf)
            
            Kunden.remove(jOpt) #Entferne den eigesetzten Kunden aus der Liste der noch ausstehenden Kunden
        
        else:
            reject = copy.deepcopy(Kunden)
            Kunden = []
   


GANTT CHART

In [ ]:
def convert_to_datetime(x):
      return datetime.fromtimestamp(31536000+x*24*3600).strftime("%Y-%m-%d")

In [ ]:
start = copy.deepcopy(startzeitfenster[:-1])
ende = copy.deepcopy(endezeitfenster[:-1])

In [ ]:
for j in range(len(start)):
    start[j] = convert_to_datetime(start[j])
    ende[j] = convert_to_datetime(ende[j])

In [ ]:
num_tick_labels = np.linspace(start = 0, stop = 100, num = 21, dtype = int)
date_ticks = [convert_to_datetime(x) for x in num_tick_labels]

In [ ]:
df = pd.DataFrame({'Kunde': I, "Start": start , "Finish": ende})

In [ ]:
fig = ff.create_gantt(df)
fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
for i in range(1, 0, -1):
    print(i)

In [ ]:
import plotly.express as px
fig2 = px.timeline(df, x_start="Start", x_end="Finish", y="Kunde")
fig2.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig2.update_yaxes(autorange="reversed")
fig2.show()